In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from tensorflow.keras import layers
import itertools
import random

# Data Wrangling

## Gathering Data

### Dataset Makanan

In [2]:
foods_df = pd.read_csv("/kaggle/input/data-makanan-stunting/foods.csv", )
foods_df

,Unnamed: 0,Menu,Energy (kJ),Protein (g),Fat (g),Carbohydrates (g),Dietary Fiber (g),PUFA (g),Cholesterol (mg),Vitamin A (mg),...,Vitamin B6 (mg),Total Folic Acid (µg),Vitamin C (mg),Sodium (mg),Potassium (mg),Calcium (mg),Magnesium (mg),Phosphorus (mg),Iron (mg),Zinc (mg)
0,0,Breastmilk (> 10 days post partum),288,1.13,4.03,7.0,0.0,0.0,0.016,69.0,...,0.014,8,6.5,13,47,29,3.2,15,0.058,0.134
1,1,kemiri,1155,22.40,4.40,51.7,10.3,0.9,0.000,2334.0,...,1.000,608,122.0,452,3805,1468,249.0,351,97.900,4.800
2,2,adonan tepung beras,544,2.40,0.20,28.6,0.3,0.1,0.000,0.0,...,0.050,2,0.0,0,29,3,13.0,37,0.200,0.400
3,3,adonan tepung tapioka,1594,0.30,0.10,91.3,0.9,0.0,0.000,0.0,...,0.000,0,0.0,9,3,2,3.0,13,0.500,0.100
4,4,adonan tepung terigu,1523,10.30,1.00,76.3,2.7,0.4,0.000,0.0,...,0.040,18,0.0,2,107,15,22.0,108,1.200,0.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,1121,Sup sayuran,300,6.00,3.00,60.0,6.0,0.2,10.000,2.0,...,0.300,30,20.0,10,600,25,18.0,50,1.000,0.300
1122,1122,Bola-bola nasi dengan sayuran,310,7.00,4.00,62.0,4.5,0.3,15.000,2.0,...,0.300,40,25.0,15,700,30,20.0,55,1.200,0.500
1123,1123,Omelet sayur,300,8.00,2.00,50.0,3.0,0.4,5.000,1.5,...,0.300,25,18.0,20,500,20,15.0,40,0.900,0.400
1124,1124,Sup ayam dan kentang,260,4.00,1.00,30.0,5.0,0.3,0.000,1.0,...,0.200,20,15.0,15,400,15,12.0,30,0.600,0.300


### Dataset Standar Nutrisi Stunting

In [3]:
standard_nutrition_df = pd.read_csv("/kaggle/input/data-makanan-stunting/standard-nutrition.csv")
standard_nutrition_df

,Unnamed: 0,Nutrisi,Minimum,Maximum,Dampak Kelebihan,Dampak Kekurangan,Fungsi Zat,Rekomendasi Harian Anak (1-5 tahun)
0,0,Energy (kJ),4000.0,5000.0,"Obesitas, penyakit jantung, diabetes","Kelelahan, penurunan berat badan",Sumber energi bagi tubuh,4000-5000 kJ
1,1,Protein (g),13.0,20.0,"Gangguan ginjal, dehidrasi","Stunting, sistem imun yang lemah","Memperbaiki jaringan, pertumbuhan otot",13-20 g
2,2,Fat (g),NaN,NaN,"Penyakit jantung, obesitas","Kulit kering, gangguan penyerapan vitamin","Sumber energi, penyerapan vitamin larut lemak",30-35% of total daily calories
3,3,Carbohydrates (g),130.0,130.0,"Obesitas, diabetes tipe 2","Kelelahan, gangguan fungsi otak",Sumber energi utama,130 g
4,4,Dietary Fiber (g),14.0,19.0,"Gangguan pencernaan, kembung","Sembelit, risiko kanker usus besar",Menjaga kesehatan pencernaan,14-19 g
5,5,PUFA (g),4.0,7.0,"Gangguan perdarahan, risiko kanker","Gangguan fungsi otak, kesehatan jantung","Meningkatkan kesehatan jantung, peradangan",4-7 g
6,6,Cholesterol (mg),NaN,NaN,"Penyakit jantung, stroke",Kesehatan jantung terganggu,"Komponen membran sel, produksi hormon",100 mg max
7,7,Vitamin A (mg),NaN,NaN,"Mual, gangguan hati, peningkatan tekanan intra...","Kebutaan malam, sistem imun lemah",Penting untuk penglihatan dan fungsi sistem imun,0.3-0.4 mg (300-400 mcg)
8,8,Vitamin E (eq.) (mg),5.0,7.0,"Diare, mual","Kerusakan sel, masalah reproduksi","Antioksidan, melindungi sel dari kerusakan",5-7 mg
9,9,Vitamin B1 (mg),NaN,NaN,"Kelelahan, gangguan sistem saraf","Beri-Beri, kelemahan otot",Metabolisme energi,0.5 mg


## Assessing Data

### Data Makanan

In [4]:
foods_df.info()
print("\nDuplicated Data: {}\n\nMissing value:\n{}".format(foods_df.duplicated().sum(), foods_df.isna().sum()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1126 non-null   int64  
 1   Menu                   1126 non-null   object 
 2   Energy (kJ)            1126 non-null   int64  
 3   Protein (g)            1126 non-null   float64
 4   Fat (g)                1126 non-null   float64
 5   Carbohydrates (g)      1126 non-null   float64
 6   Dietary Fiber (g)      1126 non-null   float64
 7   PUFA (g)               1126 non-null   float64
 8   Cholesterol (mg)       1126 non-null   float64
 9   Vitamin A (mg)         1126 non-null   float64
 10  Vitamin E (eq.) (mg)   1126 non-null   float64
 11  Vitamin B1 (mg)        1126 non-null   float64
 12  Vitamin B2 (mg)        1126 non-null   float64
 13  Vitamin B6 (mg)        1126 non-null   float64
 14  Total Folic Acid (µg)  1126 non-null   int64  
 15  Vita

## Data Standar Nutrisi

In [5]:
standard_nutrition_df.info()
print("\nDuplicated Data: {}\n\nMissing value:\n{}".format(standard_nutrition_df.duplicated().sum(), standard_nutrition_df.isna().sum()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           21 non-null     int64  
 1   Nutrisi                              21 non-null     object 
 2   Minimum                              15 non-null     float64
 3   Maximum                              15 non-null     float64
 4   Dampak Kelebihan                     21 non-null     object 
 5   Dampak Kekurangan                    21 non-null     object 
 6   Fungsi Zat                           21 non-null     object 
 7   Rekomendasi Harian Anak (1-5 tahun)  21 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 1.4+ KB

Duplicated Data: 0

Missing value:
Unnamed: 0                             0
Nutrisi                                0
Minimum                                6
Maximum          

# Pembuatan Model Rekomendasi

In [6]:
# Membuat label unik untuk setiap makanan
foods_df['label'] = foods_df.index  # Label numerik untuk setiap makanan

# Menggunakan TF-IDF pada kolom nama makanan
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(foods_df['Menu']).toarray()  # Fitur TF-IDF
y = foods_df['label'].values  # Label

In [7]:
# Membangun model
model = keras.Sequential([
    layers.Input(shape=(X.shape[1],)),  # Input: dimensi fitur TF-IDF
    layers.Dense(64, activation='relu'), # Lapisan tersembunyi
    layers.Dense(32, activation='relu'), # Lapisan tersembunyi
    layers.Dense(len(foods_df), activation='softmax')  # Output: jumlah kelas makanan
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan seluruh dataset
model.fit(X, y, epochs=50)  # Sesuaikan jumlah epoch sesuai kebutuhan


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - loss: 7.0300
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0046 - loss: 7.0216   
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0255 - loss: 6.9991
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0465 - loss: 6.9462
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0399 - loss: 6.8387
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0505 - loss: 6.6755
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0708 - loss: 6.4407
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1319 - loss: 6.0932
Epoch 9/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2693 - loss: 5.6637
Epoch 10/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4013 - loss: 5.1523
Epoch 11/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5351 - loss: 4.5757
Epoch 12/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.64

In [8]:
def detect_food_name(input_foods, model, tfidf, dataset):
    # Mengonversi input makanan ke TF-IDF
    input_features = tfidf.transform(input_foods).toarray()
    
    # Memprediksi makanan
    predictions = model.predict(input_features)
    predicted_classes = [all_food_df.iloc[np.argmax(pred)]["Menu"] for pred in predictions]
    
    # Menampilkan hasil prediksi
    for i, food in enumerate(input_foods):
        print(f"{food} terdeteksi sebagai: {predicted_classes[i]}")

In [9]:
import itertools

def evaluate_nutrition(total_nutrition, standard_nutrition):
    evaluation = {}
    deficiencies = {}

    for nutrient_row in standard_nutrition.itertuples():
        nutrient = nutrient_row.Nutrisi
        minimum = nutrient_row.Minimum
        maximum = nutrient_row.Maximum
        
        # Access the nutrient value in total_nutrition
        if nutrient in total_nutrition.columns:
            total = total_nutrition.at[0, nutrient]
        else:
            total = np.nan

        # Determine the correct unit based on the nutrient name
        if "(kJ)" in nutrient:
            unit = " kJ"
        elif "(mg)" in nutrient:
            unit = " mg"
        elif "(µg)" in nutrient:
            unit = " µg"
        elif "(g)" in nutrient:
            unit = " g"
        else:
            unit = ""  # Default to empty if no specific unit is detected

        # Evaluate nutrient levels with correct units
        if not np.isnan(total):
            if total < minimum:
                evaluation[nutrient] = f"Deficiency of {minimum - total}{unit}"
                deficiencies[nutrient] = minimum - total
            elif total > maximum:
                evaluation[nutrient] = f"Excess of {total - maximum}{unit}"
            else:
                evaluation[nutrient] = "Sufficient"
        else:
            evaluation[nutrient] = "Data unavailable"
    
    return evaluation, deficiencies

In [10]:
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [11]:

def evaluate_combination(combination, food_nutrient_profiles, standard_nutrition, threshold):
    total_nutrition = {nutrient: 0 for nutrient in standard_nutrition['Nutrisi']}
    for food_name in combination:
        food_nutrients = food_nutrient_profiles[food_name]
        for nutrient in total_nutrition:
            total_nutrition[nutrient] += food_nutrients.get(nutrient, 0)

    # Check if combination meets the 90% threshold
    for _, row in standard_nutrition.iterrows():
        nutrient = row['Nutrisi']
        minimum = row['Minimum']
        if not pd.isna(minimum) and total_nutrition.get(nutrient, 0) < minimum * threshold:
            return None  # Early exit if threshold not met

    return {
        "Food 1": combination[0],
        "Food 2": combination[1],
        "Food 3": combination[2],
        "Total Nutrition": total_nutrition
    }

In [12]:
def generate_max_combinations(foods_df, standard_nutrition, initial_threshold=0.9, num_clusters=8, max_attempts=4):
    # Define keywords for ingredients to exclude
    exclude_keywords = ["garam", "gula", "lada", "asam", "cuka", "santan", "penyedap", "bumbu", "kecap", "saus", "minyak", "larutan"]

    # Filter out rows where 'Menu' contains any of the excluded keywords
    foods_df_filtered = foods_df[~foods_df["Menu"].str.lower().str.contains('|'.join(exclude_keywords))]

    # Precompute nutrient profiles for each food in the filtered DataFrame
    food_nutrient_profiles = {food: foods_df_filtered[foods_df_filtered["Menu"] == food].iloc[0].to_dict() for food in foods_df_filtered["Menu"].unique()}

    # Cluster foods to ensure variety in nutrient profiles
    nutrient_data = foods_df_filtered.drop(columns=["Menu"]).fillna(0)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(nutrient_data)
    foods_df_filtered['Cluster'] = kmeans.labels_

    # Store foods by clusters for diverse selections
    cluster_groups = [foods_df_filtered[foods_df_filtered['Cluster'] == i]["Menu"].tolist() for i in range(num_clusters)]

    # Track all valid combinations across all passes
    all_combinations = []

    for attempt in range(max_attempts):
        # Decrease the threshold with each attempt to allow more combinations
        current_threshold = initial_threshold - (attempt * 0.05)

        # Generate a set of unique combinations by selecting from each cluster
        sampled_combinations = set()
        while len(sampled_combinations) < len(cluster_groups) ** 3:  # Theoretical max combinations
            combination = tuple(random.choice(cluster) for cluster in cluster_groups if cluster)
            sampled_combinations.add(combination)

        # Evaluate each combination and add valid ones
        for combination in sampled_combinations:
            total_nutrition = {nutrient: 0 for nutrient in standard_nutrition['Nutrisi']}
            for food_name in combination:
                food_nutrients = food_nutrient_profiles[food_name]
                for nutrient in total_nutrition:
                    total_nutrition[nutrient] += food_nutrients.get(nutrient, 0)

            # Check if the combination meets the current threshold
            meets_threshold = True
            for _, row in standard_nutrition.iterrows():
                nutrient = row['Nutrisi']
                minimum = row['Minimum']
                if not pd.isna(minimum) and total_nutrition.get(nutrient, 0) < minimum * current_threshold:
                    meets_threshold = False
                    break

            # If the combination meets the threshold, add it to the list
            if meets_threshold:
                all_combinations.append({
                    "Food 1": combination[0],
                    "Food 2": combination[1],
                    "Food 3": combination[2],
                    "Total Nutrition": total_nutrition,
                    "Meets Threshold": meets_threshold
                })

    # Convert all unique valid combinations to a DataFrame
    combinations_df = pd.DataFrame(all_combinations).drop_duplicates(subset=["Food 1", "Food 2", "Food 3"])

    return combinations_df

In [13]:
def choose_random_combination(combinations_df):
    if not combinations_df.empty:
        random_choice = combinations_df.sample(n=1).iloc[0]
        recommendation_summary = "### Randomly Selected Food Combination:\n"
        recommendation_summary += f"- {random_choice['Food 1']}\n"
        recommendation_summary += f"- {random_choice['Food 2']}\n"
        recommendation_summary += f"- {random_choice['Food 3']}\n"
        recommendation_summary += "\nTotal Nutrition:\n"
        for nutrient, value in random_choice["Total Nutrition"].items():
            recommendation_summary += f"{nutrient}: {value}\n"
        return recommendation_summary
    else:
        return "No valid combinations found that meet the nutritional threshold."

# Example usage:
# Assume `foods_df` and `standard_nutrition_df` are already defined
combinations_df = generate_max_combinations(foods_df, standard_nutrition_df, initial_threshold=0.9, num_clusters=8, max_attempts=30)
print("Generated Combinations DataFrame:\n", combinations_df)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/tmp/ipykernel_30/1343717557.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  foods_df_filtered['Cluster'] = kmeans.labels_


Generated Combinations DataFrame:
                       Food 1                Food 2  \
0       lodeh tahu and tempe             hati ayam   
1                  soto ayam  hati ayam rebus belu   
2            ikan teri segar             hati ayam   
3                  soto ayam             hati ayam   
4                 bika ambon          hati menthok   
...                      ...                   ...   
10378      es mambo/es lilin             hati itik   
10379              sengkulun          hati menthok   
10380  bubur sun beras merah           hati burung   
10381           Puree wortel       hati itik bacem   
10382                vanilli             hati itik   

                            Food 3  \
0              tumis kangkung belu   
1            daun labu siam mentah   
2                      pisang raja   
3                  gembili / mbolo   
4                      jeruk nipis   
...                            ...   
10378                 mangga sepat   
10379       

In [14]:
combinations_df.to_csv("/kaggle/working/combinations-food-recommend.csv")
combinations_df

,Food 1,Food 2,Food 3,Total Nutrition,Meets Threshold
0,lodeh tahu and tempe,hati ayam,tumis kangkung belu,"{'Energy (kJ)': 7129, 'Protein (g)': 200.29999...",True
1,soto ayam,hati ayam rebus belu,daun labu siam mentah,"{'Energy (kJ)': 7239, 'Protein (g)': 183.89999...",True
2,ikan teri segar,hati ayam,pisang raja,"{'Energy (kJ)': 8134, 'Protein (g)': 255.49999...",True
3,soto ayam,hati ayam,gembili / mbolo,"{'Energy (kJ)': 7552, 'Protein (g)': 212.29999...",True
4,bika ambon,hati menthok,jeruk nipis,"{'Energy (kJ)': 7995, 'Protein (g)': 220.5, 'F...",True
...,...,...,...,...,...
10378,es mambo/es lilin,hati itik,mangga sepat,"{'Energy (kJ)': 5769, 'Protein (g)': 175.89999...",True
10379,sengkulun,hati menthok,daun singkong mentah,"{'Energy (kJ)': 7833, 'Protein (g)': 175.7, 'F...",True
10380,bubur sun beras merah,hati burung,nasi tim,"{'Energy (kJ)': 6670, 'Protein (g)': 183.29999...",True
10381,Puree wortel,hati itik bacem,daging burung bagian kepala,"{'Energy (kJ)': 7973, 'Protein (g)': 232.1, 'F...",True


In [15]:
def choose_three_random_combinations(combinations_df):
    # Check if there are enough combinations to select three
    if combinations_df.shape[0] < 3:
        return "Not enough valid combinations to provide three unique recommendations."

    # Randomly sample three unique combinations
    random_combinations = combinations_df.sample(n=3).reset_index(drop=True)

    # Create a summary for each selected combination
    recommendation_summary = "### Three Randomly Selected Food Combinations:\n"
    for i in range(3):
        recommendation_summary += f"\n**Combination {i + 1}:**\n"
        recommendation_summary += f"- {random_combinations.loc[i, 'Food 1']}\n"
        recommendation_summary += f"- {random_combinations.loc[i, 'Food 2']}\n"
        recommendation_summary += f"- {random_combinations.loc[i, 'Food 3']}\n"
        recommendation_summary += "\nTotal Nutrition:\n"
        for nutrient, value in random_combinations.loc[i, "Total Nutrition"].items():
            recommendation_summary += f"{nutrient}: {value}\n"

    return recommendation_summary

In [16]:
def generate_summary(evaluation, standard_nutrition):
    summary = ""
    recommendations = {
        "increase": [],
        "moderate": [],
        "maintain": []
    }

    # Convert standard_nutrition DataFrame into a dictionary for quick lookup
    nutrient_info = {
        row['Nutrisi']: {
            "excess_impact": row["Dampak Kelebihan"],
            "deficiency_impact": row["Dampak Kekurangan"],
            "function": row["Fungsi Zat"]
        } for _, row in standard_nutrition.iterrows()
    }

    # Categorize nutrients based on their evaluation status
    for nutrient, status in evaluation.items():
        if "Deficiency" in status:
            recommendations["increase"].append(nutrient)
        elif "Excess" in status:
            recommendations["moderate"].append(nutrient)
        elif status == "Sufficient":
            recommendations["maintain"].append(nutrient)
    
    # Generate the summary
    summary += "### Summary of Nutritional Status:\n"
    
    # Sufficient Nutrients
    if recommendations["maintain"]:
        summary += f"\nSufficient Nutrients:\n"
        summary += ", ".join(recommendations["maintain"]) + " levels are adequate, indicating the current diet meets the minimum requirements for these nutrients.\n"
    
    # Excess Nutrients
    if recommendations["moderate"]:
        summary += f"\nExcess Nutrients:\n"
        for nutrient in recommendations["moderate"]:
            impact = nutrient_info.get(nutrient, {}).get("excess_impact", "Consider reducing intake to avoid potential health risks.")
            summary += f"- {nutrient}: {impact}\n"
    
    # Deficient Nutrients
    if recommendations["increase"]:
        summary += f"\nDeficient Nutrients:\n"
        for nutrient in recommendations["increase"]:
            impact = nutrient_info.get(nutrient, {}).get("deficiency_impact", "Increase intake to meet daily requirements.")
            summary += f"- {nutrient}: {impact}\n"

    # Generate Overall Recommendations
    summary += "\n### Recommendations:\n"
    if recommendations["increase"]:
        summary += "- **Increase** foods rich in: " + ", ".join(recommendations["increase"]) + " to meet daily requirements.\n"
    if recommendations["moderate"]:
        summary += "- **Moderate** intake of: " + ", ".join(recommendations["moderate"]) + " to avoid potential excesses.\n"
    if recommendations["maintain"]:
        summary += "- **Maintain** current levels of: " + ", ".join(recommendations["maintain"]) + ".\n"
    
    return summary


In [17]:
def detect_and_evaluate(input_foods, model, tfidf, dataset, standard_nutrition):
    # Convert input foods to TF-IDF features
    input_features = tfidf.transform(input_foods).toarray()
    
    # Predict food names
    predictions = model.predict(input_features)
    predicted_classes = [dataset.iloc[np.argmax(pred)] for pred in predictions]
    
    # Initialize total_nutrition with zero for each nutrient column in standard_nutrition
    total_nutrition = {nutrient: 0 for nutrient in standard_nutrition['Nutrisi'] if nutrient in dataset.columns}

    # Sum nutrition values for detected foods
    for food in predicted_classes:
        for nutrient in total_nutrition.keys():
            total_nutrition[nutrient] += food.get(nutrient, 0)  # Safely sum nutrient values

    # Convert total nutrition to a DataFrame for evaluation
    total_nutrition_df = pd.DataFrame([total_nutrition])

    # Evaluate nutrition
    evaluation, deficiencies = evaluate_nutrition(total_nutrition_df, standard_nutrition)
    
    # Display detected food names and their nutrition
    for food in predicted_classes:
        nutrients = [food.get(nutrient, 'N/A') for nutrient in total_nutrition.keys()]
        print(f"{food['Menu']} detected with nutrition: {nutrients}")

    print("\nTotal Nutrition from detected foods:")
    print(total_nutrition_df)

    # Display nutrition evaluation
    print("\nNutrition Evaluation:")
    for nutrient, status in evaluation.items():
        print(f"{nutrient}: {status}")

    summary = generate_summary(evaluation, standard_nutrition_df)
    print(summary)
    recommendation_food = choose_three_random_combinations(combinations_df)
    print(recommendation_food)

In [22]:
# Input makanan dari pengguna
food_1 = "sayur daging"
food_2 = "sayur sop"
food_3 = "sayur lodeh"
input_foods = [food_1, food_2, food_3]

# Melakukan deteksi dan evaluasi
detect_and_evaluate(input_foods, model, tfidf, foods_df, standard_nutrition_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
sayur gulai daging detected with nutrition: [1230, 17.6, 23.3, 3.4, 1.9, 1.1, 61.0, 5.0, 0.0, 0.06, 0.11, 0.29, 19, 1.0, 39, 317, 27, 29.0, 152, 2.7, 3.0]
sayur sop detected with nutrition: [435, 1.8, 7.0, 10.5, 3.3, 0.2, 0.0, 473.0, 1.0, 0.07, 0.05, 0.1, 32, 3.0, 28, 213, 23, 24.0, 55, 1.1, 0.5]
sayur lodeh detected with nutrition: [234, 1.9, 3.4, 6.0, 1.7, 0.6, 0.0, 80.0, 2.0, 0.04, 0.07, 0.05, 26, 13.0, 5, 171, 38, 25.0, 48, 0.6, 0.3]

Total Nutrition from detected foods:
   Energy (kJ)  Protein (g)  Fat (g)  Carbohydrates (g)  Dietary Fiber (g)  \
0         1899         21.3     33.7               19.9                6.9   

   PUFA (g)  Cholesterol (mg)  Vitamin A (mg)  Vitamin E (eq.) (mg)  \
0       1.9              61.0           558.0                   3.0   

   Vitamin B1 (mg)  ...  Vitamin B6 (mg)  Total Folic Acid (µg)  \
0             0.17  ...             0.44                     77   

   Vitamin C (mg)  Sodium (mg)  Potassium (mg) 